In [1]:
# MLP complex neural network on processed data 2

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# === CONFIG ===
data_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 20
batch_size = 64

# === Load File List and Labels ===
npz_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.npz')])

all_labels = []
for file in npz_files:
    y = np.load(file)['y']
    all_labels.extend(y)
all_labels = np.array(all_labels)

classes = np.unique(all_labels)
class_weights_array = compute_class_weight(class_weight='balanced', classes=classes, y=all_labels)
class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32).to(device)

# === Train-Test Split ===
train_files, test_files = train_test_split(npz_files, test_size=0.2, random_state=42, shuffle=True)

# === Fit Scaler on Training Data ===
scaler = StandardScaler()
sample_data = []
for file in train_files[:3]:
    X = np.load(file)['X']
    X = X.reshape(X.shape[0], -1)
    sample_data.append(X)
scaler.fit(np.vstack(sample_data))

# === Define a More Complex Model ===
class ComplexEEGNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ComplexEEGNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# === Train the Model ===
model = ComplexEEGNet(input_dim=sample_data[0].shape[1], num_classes=len(classes)).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === Training Loop ===
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for file in train_files:
        data = np.load(file)
        X = data['X'].reshape(data['X'].shape[0], -1)
        y = data['y']
        X = scaler.transform(X)

        X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(y, dtype=torch.long).to(device)

        for start in range(0, len(X_tensor), batch_size):
            end = start + batch_size
            X_batch = X_tensor[start:end]
            y_batch = y_tensor[start:end]

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

# === Evaluation Function ===
def evaluate(files, label="Test"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for file in files:
            data = np.load(file)
            X = data['X'].reshape(data['X'].shape[0], -1)
            y = data['y']
            X = scaler.transform(X)

            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            outputs = model(X_tensor)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()

            y_true.extend(y)
            y_pred.extend(predictions)

    acc = np.mean(np.array(y_true) == np.array(y_pred))
    print(f"\n📊 {label} Accuracy: {acc:.4f}")
    print(classification_report(y_true, y_pred, target_names=['pronation', 'supination']))
    return acc

# === Final Evaluation ===
train_acc = evaluate(train_files, "Train")
test_acc = evaluate(test_files, "Test")


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20

📊 Train Accuracy: 0.9357
              precision    recall  f1-score   support

   pronation       0.92      0.96      0.94      2800
  supination       0.95      0.92      0.93      2800

    accuracy                           0.94      5600
   macro avg       0.94      0.94      0.94      5600
weighted avg       0.94      0.94      0.94      5600


📊 Test Accuracy: 0.5050
              precision    recall  f1-score   support

   pronation       0.50      0.54      0.52       700
  supination       0.51      0.47      0.49       700

    accuracy                           0.51      1400
   macro avg       0.51      0.51      0.50      1400
weighted avg       0.51      0.51      0.50      1400



In [2]:
# fully connected MLP neural network

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc
)

# === CONFIG ===
data_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 20
batch_size = 64

# === Load Files and Labels ===
npz_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.npz')])

all_labels = []
for file in npz_files:
    y = np.load(file)['y']
    all_labels.extend(y)
all_labels = np.array(all_labels)

classes = np.unique(all_labels)
class_weights_array = compute_class_weight(class_weight='balanced', classes=classes, y=all_labels)
class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32).to(device)

train_files, test_files = train_test_split(npz_files, test_size=0.2, random_state=42, shuffle=True)

# === Fit Scaler ===
scaler = StandardScaler()
sample_data = []
for file in train_files[:3]:
    X = np.load(file)['X']
    X = X.reshape(X.shape[0], -1)
    sample_data.append(X)
scaler.fit(np.vstack(sample_data))

# === Model ===
class ComplexEEGNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ComplexEEGNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# === Initialize Model ===
input_dim = sample_data[0].shape[1]
model = ComplexEEGNet(input_dim=input_dim, num_classes=len(classes)).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# === Train Model ===
train_losses = []
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for file in train_files:
        data = np.load(file)
        X = data['X'].reshape(data['X'].shape[0], -1)
        y = data['y']
        X = scaler.transform(X)

        X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(y, dtype=torch.long).to(device)

        for start in range(0, len(X_tensor), batch_size):
            end = start + batch_size
            X_batch = X_tensor[start:end]
            y_batch = y_tensor[start:end]

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
    avg_loss = epoch_loss / len(train_files)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

# === Evaluation Function ===
def evaluate(files, label="Test"):
    model.eval()
    y_true, y_pred, y_scores = [], [], []
    with torch.no_grad():
        for file in files:
            data = np.load(file)
            X = data['X'].reshape(data['X'].shape[0], -1)
            y = data['y']
            X = scaler.transform(X)

            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            outputs = model(X_tensor)
            probs = torch.softmax(outputs, dim=1)
            predictions = torch.argmax(probs, dim=1).cpu().numpy()

            y_true.extend(y)
            y_pred.extend(predictions)
            y_scores.extend(probs[:, 1].cpu().numpy())  # For ROC AUC (class 1)

    acc = np.mean(np.array(y_true) == np.array(y_pred))
    print(f"\n📊 {label} Accuracy: {acc:.4f}")
    print(classification_report(y_true, y_pred, target_names=['Pronation', 'Supination']))
    return np.array(y_true), np.array(y_pred), np.array(y_scores)

# === Final Evaluation ===
y_true_train, y_pred_train, _ = evaluate(train_files, "Train")
y_true_test, y_pred_test, y_scores_test = evaluate(test_files, "Test")

# === Confusion Matrix ===
cm = confusion_matrix(y_true_test, y_pred_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Pronation', 'Supination'])

# === ROC Curve ===
fpr, tpr, _ = roc_curve(y_true_test, y_scores_test)
roc_auc = auc(fpr, tpr)

# === Plotting ===
plt.figure(figsize=(18, 5))

# Loss Curve
plt.subplot(1, 3, 1)
plt.plot(range(1, epochs+1), train_losses, marker='o')
plt.title("Training Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)

# ROC Curve
plt.subplot(1, 3, 2)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(True)

# Confusion Matrix
plt.subplot(1, 3, 3)
disp.plot(ax=plt.gca(), cmap='Blues', colorbar=False)
plt.title("Confusion Matrix Heatmap")

plt.tight_layout()
plt.show()


Epoch 1/20 - Loss: 1.3973
Epoch 2/20 - Loss: 1.3828
Epoch 3/20 - Loss: 1.3811
Epoch 4/20 - Loss: 1.3697
Epoch 5/20 - Loss: 1.3550
Epoch 6/20 - Loss: 1.3390
Epoch 7/20 - Loss: 1.3107
Epoch 8/20 - Loss: 1.2985
Epoch 9/20 - Loss: 1.2741
Epoch 10/20 - Loss: 1.2417
Epoch 11/20 - Loss: 1.2335
Epoch 12/20 - Loss: 1.2083
Epoch 13/20 - Loss: 1.1877
Epoch 14/20 - Loss: 1.1661


TypeError: '_thread.RLock' object cannot be interpreted as an integer

In [3]:
# MLP complex neural network on processed data 2

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# === CONFIG ===
data_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data3'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 20
batch_size = 64

# === Load File List and Labels ===
npz_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.npz')])

all_labels = []
for file in npz_files:
    y = np.load(file)['y']
    all_labels.extend(y)
all_labels = np.array(all_labels)

classes = np.unique(all_labels)
class_weights_array = compute_class_weight(class_weight='balanced', classes=classes, y=all_labels)
class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32).to(device)

# === Train-Test Split ===
train_files, test_files = train_test_split(npz_files, test_size=0.2, random_state=42, shuffle=True)

# === Fit Scaler on Training Data ===
scaler = StandardScaler()
sample_data = []
for file in train_files[:3]:
    X = np.load(file)['X']
    X = X.reshape(X.shape[0], -1)
    sample_data.append(X)
scaler.fit(np.vstack(sample_data))

# === Define a More Complex Model ===
class ComplexEEGNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ComplexEEGNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# === Train the Model ===
model = ComplexEEGNet(input_dim=sample_data[0].shape[1], num_classes=len(classes)).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === Training Loop ===
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for file in train_files:
        data = np.load(file)
        X = data['X'].reshape(data['X'].shape[0], -1)
        y = data['y']
        X = scaler.transform(X)

        X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(y, dtype=torch.long).to(device)

        for start in range(0, len(X_tensor), batch_size):
            end = start + batch_size
            X_batch = X_tensor[start:end]
            y_batch = y_tensor[start:end]

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

# === Evaluation Function ===
def evaluate(files, label="Test"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for file in files:
            data = np.load(file)
            X = data['X'].reshape(data['X'].shape[0], -1)
            y = data['y']
            X = scaler.transform(X)

            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            outputs = model(X_tensor)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()

            y_true.extend(y)
            y_pred.extend(predictions)

    acc = np.mean(np.array(y_true) == np.array(y_pred))
    print(f"\n📊 {label} Accuracy: {acc:.4f}")
    print(classification_report(y_true, y_pred, target_names=['pronation', 'supination']))
    return acc

# === Final Evaluation ===
train_acc = evaluate(train_files, "Train")
test_acc = evaluate(test_files, "Test")


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20

📊 Train Accuracy: 0.9940
              precision    recall  f1-score   support

   pronation       0.99      1.00      0.99      2578
  supination       1.00      0.99      0.99      2576

    accuracy                           0.99      5154
   macro avg       0.99      0.99      0.99      5154
weighted avg       0.99      0.99      0.99      5154


📊 Test Accuracy: 0.5065
              precision    recall  f1-score   support

   pronation       0.51      0.51      0.51       693
  supination       0.51      0.50      0.50       691

    accuracy                           0.51      1384
   macro avg       0.51      0.51      0.51      1384
weighted avg       0.51      0.51      0.51      1384



In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split

# === CONFIG ===
data_dir = '/storage/projects1/e19-4yp-mi-eeg-for-bci/ashan/processed_data3'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 64
patience = 7  # Early stopping patience

# === Custom Dataset to load npz files chunk-wise ===
class EEGDataset(Dataset):
    def __init__(self, files, scaler=None):
        self.files = files
        self.scaler = scaler
        self.data = []
        self.labels = []
        self._load_all()

    def _load_all(self):
        # Load all data into memory - for smaller datasets
        # If dataset is too large, consider loading batch-wise from disk instead
        all_X, all_y = [], []
        for f in self.files:
            data = np.load(f)
            X = data['X'].reshape(data['X'].shape[0], -1)
            y = data['y']
            all_X.append(X)
            all_y.append(y)
        X = np.vstack(all_X)
        y = np.hstack(all_y)
        if self.scaler:
            X = self.scaler.transform(X)
        self.data = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# === Load File List and Labels ===
npz_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.npz')])

all_labels = []
for file in npz_files:
    y = np.load(file)['y']
    all_labels.extend(y)
all_labels = np.array(all_labels)

classes = np.unique(all_labels)
class_weights_array = compute_class_weight(class_weight='balanced', classes=classes, y=all_labels)
class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32).to(device)

# === Split data into train, validation, test sets ===
train_val_files, test_files = train_test_split(npz_files, test_size=0.2, random_state=42, shuffle=True)
train_files, val_files = train_test_split(train_val_files, test_size=0.2, random_state=42, shuffle=True)

# === Fit Scaler on Training Data ===
scaler = StandardScaler()
sample_data = []
for file in train_files:
    X = np.load(file)['X'].reshape(-1, np.load(file)['X'].shape[1]*np.load(file)['X'].shape[2])
    sample_data.append(X)
scaler.fit(np.vstack(sample_data))

# === Create Dataset and DataLoader ===
train_dataset = EEGDataset(train_files, scaler=scaler)
val_dataset = EEGDataset(val_files, scaler=scaler)
test_dataset = EEGDataset(test_files, scaler=scaler)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# === Define Model ===
class ComplexEEGNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ComplexEEGNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.4),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# === Initialize Model, Loss, Optimizer, Scheduler ===
input_dim = train_dataset.data.shape[1]
model = ComplexEEGNet(input_dim=input_dim, num_classes=len(classes)).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Weight decay for L2 regularization
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# === Training with Early Stopping ===
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    train_loss /= total
    train_acc = correct / total

    # Validation phase
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            val_loss += loss.item() * X_batch.size(0)
            preds = outputs.argmax(dim=1)
            val_correct += (preds == y_batch).sum().item()
            val_total += y_batch.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Epoch {epoch + 1}/{epochs} — Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f} — Val loss: {val_loss:.4f}, Val acc: {val_acc:.4f}")

    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# === Load best model for testing ===
model.load_state_dict(torch.load('best_model.pt'))

# === Evaluation function ===
def evaluate(loader, label="Test"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            preds = outputs.argmax(dim=1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    acc = np.mean(np.array(y_true) == np.array(y_pred))
    print(f"\n📊 {label} Accuracy: {acc:.4f}")
    print(classification_report(y_true, y_pred, target_names=['pronation', 'supination']))
    return acc

# === Final evaluation ===
train_acc = evaluate(train_loader, "Train")
val_acc = evaluate(val_loader, "Validation")
test_acc = evaluate(test_loader, "Test")


/storage/projects1/e19-4yp-mi-eeg-for-bci/env/eeg-env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/50 — Train loss: 0.7296, Train acc: 0.5046 — Val loss: 0.6991, Val acc: 0.4899
Epoch 2/50 — Train loss: 0.6983, Train acc: 0.5370 — Val loss: 0.7027, Val acc: 0.4855
Epoch 3/50 — Train loss: 0.6688, Train acc: 0.5810 — Val loss: 0.7140, Val acc: 0.4925
Epoch 4/50 — Train loss: 0.6451, Train acc: 0.6341 — Val loss: 0.7327, Val acc: 0.4811
Epoch 5/50 — Train loss: 0.6033, Train acc: 0.6848 — Val loss: 0.7697, Val acc: 0.4820
Epoch 6/50 — Train loss: 0.5321, Train acc: 0.7334 — Val loss: 0.8176, Val acc: 0.4820
Epoch 7/50 — Train loss: 0.4765, Train acc: 0.7769 — Val loss: 0.9040, Val acc: 0.4758
Epoch 8/50 — Train loss: 0.4357, Train acc: 0.8088 — Val loss: 0.8994, Val acc: 0.4785
Early stopping triggered after 8 epochs.

📊 Train Accuracy: 0.6236
              precision    recall  f1-score   support

   pronation       0.61      0.69      0.65      2005
  supination       0.64      0.56      0.60      2012

    accuracy                           0.62      4017
   macro avg       